## Basics

- 参考
    - https://huggingface.co/blog/llama2
- 4k tokens of context
- 几个版本：3\*2\*2 == 12
    - https://huggingface.co/meta-llama
    - 7B，13B，70B
    - 是否是 hf
    - 是否是 chat
        - meta-llama/Llama-2-7b
        - meta-llama/Llama-2-7b-hf
        - meta-llama/Llama-2-7b-chat
        - meta-llama/Llama-2-7b-chat-hf

In [1]:
import subprocess
import os
os.environ['CURL_CA_BUNDLE'] = ''

result = subprocess.run('bash -c "source /etc/network_turbo && env | grep proxy"', shell=True, capture_output=True, text=True)
output = result.stdout
for line in output.splitlines():
    if '=' in line:
        var, value = line.split('=', 1)
        os.environ[var] = value

## Text-generation

In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import transformers
from transformers import BitsAndBytesConfig
import torch

In [10]:
model_name = "meta-llama/Llama-2-7b-chat-hf"

In [4]:
# import os
# os.environ['HTTP_PROXY'] = 'http://127.0.0.1:7890'
# os.environ['HTTPS_PROXY'] = 'http://127.0.0.1:7890'

In [13]:
save_path = "../../model_path"   # 模型存储目录

tokenizer = AutoTokenizer.from_pretrained(model_name, cache_dir=save_path)
pipeline = transformers.pipeline(
    "text-generation",
    model=model_name,
    torch_dtype=torch.float16,
    device_map="auto",
    # cache_dir=save_path
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [14]:
tokenizer

LlamaTokenizerFast(name_or_path='meta-llama/Llama-2-7b-chat-hf', vocab_size=32000, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>'}, clean_up_tokenization_spaces=False),  added_tokens_decoder={
	0: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	2: AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}

In [15]:
pipeline.model

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (v_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (up_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (down_proj): Linear(in_features=11008, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
      )
    )
    (no

In [16]:
trainable_params = 0
all_param = 0
for name, param in pipeline.model.named_parameters():
    # print(name, param.requires_grad, param.numel())
    all_param += param.numel()
    if param.requires_grad:
        trainable_params += param.numel()
print(trainable_params, all_param, trainable_params*1./all_param)

6738415616 6738415616 1.0


In [26]:
sequences = pipeline(
    'I liked "Breaking Bad" and "Band of Brothers". Do you have any recommendations of other shows I might like?\n',
    do_sample=True,
    top_k=10,
    num_return_sequences=3,
    eos_token_id=tokenizer.eos_token_id,
    max_length=200,
)

In [27]:
for seq in sequences:
    print('===============================')
    print(seq['generated_text'] + '\n\n')

I liked "Breaking Bad" and "Band of Brothers". Do you have any recommendations of other shows I might like?

Sure! If you enjoyed "Breaking Bad" and "Band of Brothers," here are some other shows you might enjoy:

1. "The Sopranos" - This HBO series is a crime drama that follows the life of a New Jersey mob boss, Tony Soprano, as he navigates the criminal underworld and deals with personal and family issues.
2. "The Wire" - This HBO series explores the drug trade in Baltimore from multiple perspectives, including law enforcement, drug dealers, and politicians. It's known for its gritty realism and complex characters.
3. "Narcos" - This Netflix series tells the true story of Pablo Escobar, the infamous Colombian drug lord, and the DE


I liked "Breaking Bad" and "Band of Brothers". Do you have any recommendations of other shows I might like?

Yes, definitely! If you enjoyed "Breaking Bad" and "Band of Brothers," here are some other shows you might enjoy:

1. "The Sopranos" - This HBO ser

In [28]:
sequences = pipeline(
    '我比较喜欢《肖申克的救赎》、《星际穿越》这样的电影，你有其他电影推荐吗？\n',
    do_sample=True,
    top_k=10,
    num_return_sequences=3,
    eos_token_id=tokenizer.eos_token_id,
    max_length=200,
)
for seq in sequences:
    print('===============================')
    print(seq['generated_text'])

我比较喜欢《肖申克的救赎》、《星际穿越》这样的电影，你有其他电影推荐吗？

As a fan of science fiction movies, I particularly enjoy "Blade Runner" and "Star Trek." Are there any other movie recommendations you could offer?
我比较喜欢《肖申克的救赎》、《星际穿越》这样的电影，你有其他电影推荐吗？

I prefer movies like "Schindler's List" and "Interstellar" more, do you have any other movie recommendations?
我比较喜欢《肖申克的救赎》、《星际穿越》这样的电影，你有其他电影推荐吗？

Hi there! 😊 I'm a big fan of sci-fi movies like "Interstellar" and "Inception". Do you have any other movie recommendations? 🤔


## Fine-tune

使用 trl 库的 SFTTrainer。具体代码可参考 trl 库中的 example 代码，位置如下。

```
pip install trl
git clone https://github.com/lvwerra/trl
python trl/examples/scripts/sft_trainer.py \
    --model_name meta-llama/Llama-2-7b-hf \
    --dataset_name timdettmers/openassistant-guanaco \
    --load_in_4bit \
    --use_peft \
    --batch_size 16 \
    --gradient_accumulation_steps 2
```

- 对于 4090，大概需要 5 个小时
    - 3 个 epochs
    - 16 * 2 的 batch size
    - (9846 * 3) / (16 * 2 * 1) == 924 个 optimization steps，即 len(dataset) * epochs / (batch_size * gradient_accumulation_steps * n_gpus)